# Import libraries

In [ ]:
# Pre-requisites. via
# https://github.com/haotian-liu/LLaVA
!git clone https://github.com/haotian-liu/LLaVA.git
%cd LLaVA
!conda create -n llava python=3.10 -y
!conda activate llava
!pip install --upgrade pip  # enable PEP 660 support
!pip install fastapi kaleido python-multipart typing-extensions==4.5.0 uvicorn
!pip install -e .

In [ ]:
import inspect
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
# Import libraries. via
# https://medium.com/supportvectors/comparing-image-to-text-models-blip-blip2-and-llava-ebd66d2eb6d8
import torch
from llava.serve.cli import load_image
from llava.constants import IMAGE_TOKEN_INDEX
from llava.model.builder import load_pretrained_model
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria

# Instantiate model

In [ ]:
# Instantiate the model. via
# https://github.com/haotian-liu/LLaVA
# https://medium.com/supportvectors/comparing-image-to-text-models-blip-blip2-and-llava-ebd66d2eb6d8
model_path = 'liuhaotian/llava-v1.5-13b'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path),
    device=device
)

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory to store inferences

In [ ]:
os.makedirs('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions', exist_ok=True)

# Import datasets

## US-Politics

In [ ]:
constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constraint22_dataset_uspolitics_test_captioned_Qwen-VL-Chat.csv')
constraint22_dataset_uspolitics_test_captioned_InternLM_XComposer = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constraint22_dataset_uspolitics_test_captioned_InternLM-XComposer.csv')
constraint22_dataset_uspolitics_test_captioned_llava = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constraint22_dataset_uspolitics_test_captioned_llava-v1.5-13b.csv')
constraint22_dataset_uspolitics_test_captioned_BLIP_2 = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constraint22_dataset_uspolitics_test_captioned_BLIP-2.csv')

In [ ]:
constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat = constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat.dropna().reset_index(drop=True)
constraint22_dataset_uspolitics_test_captioned_InternLM_XComposer = constraint22_dataset_uspolitics_test_captioned_InternLM_XComposer.dropna().reset_index(drop=True)
constraint22_dataset_uspolitics_test_captioned_llava = constraint22_dataset_uspolitics_test_captioned_llava.dropna().reset_index(drop=True)
constraint22_dataset_uspolitics_test_captioned_BLIP_2 = constraint22_dataset_uspolitics_test_captioned_BLIP_2.dropna().reset_index(drop=True)

# Set prompts and define a function to call the model

In [ ]:
def prompt_vanilla(entity):
    return inspect.cleandoc(f"""
    What is the role of {entity} in this meme?
    hero: presented in a positive light.
    villain: portrayed negatively, e.g., in an association with adverse traits like wickedness, cruelty, hypocrisy, etc.
    victim: portrayed as suffering the negative impact of someone else’s actions.
    other: not a hero, a villain, or a victim.
    Constraint: Without using any other words, answer either hero, villain, victim, other.""")

In [ ]:
def prompt_with_OCR(entity, OCR):
    return inspect.cleandoc(f"""
    Text on this meme: \"\"\"
    {OCR}
    \"\"\"
    What is the role of {entity} in this meme?
    hero: presented in a positive light.
    villain: portrayed negatively, e.g., in an association with adverse traits like wickedness, cruelty, hypocrisy, etc.
    victim: portrayed as suffering the negative impact of someone else’s actions.
    other: not a hero, a villain, or a victim.
    Constraint: Without using any other words, answer either hero, villain, victim, other.""")

In [ ]:
def prompt_with_caption(entity, caption):
    return inspect.cleandoc(f"""
    Description of this meme: \"\"\"
    {caption}
    \"\"\"
    What is the role of {entity} in this meme?
    hero: presented in a positive light.
    villain: portrayed negatively, e.g., in an association with adverse traits like wickedness, cruelty, hypocrisy, etc.
    victim: portrayed as suffering the negative impact of someone else’s actions.
    other: not a hero, a villain, or a victim.
    Constraint: Without using any other words, answer either hero, villain, victim, other.""")

In [ ]:
# Use llava-v1.5-13b for the inference. via
# https://github.com/haotian-liu/LLaVA/blob/main/llava/serve/cli.py
# https://medium.com/supportvectors/comparing-image-to-text-models-blip-blip2-and-llava-ebd66d2eb6d8
def get_prediction(image, prompt):
    image = load_image(image)
    image_tensor = image_processor([image], return_tensors='pt')['pixel_values']
    image_tensor = image_tensor.to(model.device, dtype=torch.float16)
    prompt = f'USER: <image>\n{prompt}\nASSISTANT:'
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)
    stopping_criteria = KeywordsStoppingCriteria(["</s>"], tokenizer, input_ids)
    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor,
            do_sample=True,
            temperature=0.2,
            max_new_tokens=512,
            use_cache=True,
            stopping_criteria=[stopping_criteria])
    output = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
    return output[:-4]

# Define function to clean responses

In [ ]:
def remap(x):
    x = x.lower()
    if x in {'hero', 'villain', 'victim', 'other'}:
        return x
    elif 'hero' in x and 'villain' not in x and 'victim' not in x and 'other' not in x and 'not hero' not in x and 'not a hero' not in x:
        return 'hero'
    elif 'villain' in x and 'hero' not in x and 'victim' not in x and 'other' not in x and 'not villain' not in x and 'not a villain' not in x:
        return 'villain'
    elif 'victim' in x and 'villain' not in x and 'hero' not in x and 'other' not in x and 'not victim' not in x and 'not a victim' not in x:
        return 'victim'
    elif 'other' in x and 'villain' not in x and 'victim' not in x and 'hero' not in x and 'not other' not in x and 'not an other' not in x:
        return 'other'
    else:
        return None

# Call the `get_prediction` function and save inferences

GROUNDING [ABSENT] & PROMPT [VANILLA]

In [ ]:
uspolitics_test_grounding_absent_prompt_vanilla = constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat.copy(deep=True)
uspolitics_test_grounding_absent_prompt_vanilla_images = uspolitics_test_grounding_absent_prompt_vanilla['image'].values
uspolitics_test_grounding_absent_prompt_vanilla_entities = uspolitics_test_grounding_absent_prompt_vanilla['entity'].values
uspolitics_test_grounding_absent_prompt_vanilla['prediction'] = [get_prediction(image, prompt_vanilla(entity)) for image, entity in zip(uspolitics_test_grounding_absent_prompt_vanilla_images, uspolitics_test_grounding_absent_prompt_vanilla_entities)]
uspolitics_test_grounding_absent_prompt_vanilla.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_prompt[VANILLA]_prediction[llava-v1.5-13b].csv', index=False)
uspolitics_test_grounding_absent_prompt_vanilla['prediction'] = uspolitics_test_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_vanilla['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_vanilla['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_vanilla['prediction'] = uspolitics_test_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_vanilla['role'].values, uspolitics_test_grounding_absent_prompt_vanilla['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_vanilla['role'].values, uspolitics_test_grounding_absent_prompt_vanilla['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

villain    868
hero        98
other       28
victim       1
Name: prediction, dtype: int64
0
0.22414482439093447
              precision    recall  f1-score   support

        hero       0.67      0.27      0.38       246
     villain       0.28      0.97      0.43       250
      victim       1.00      0.00      0.01       249
       other       0.36      0.04      0.07       250

    accuracy                           0.32       995
   macro avg       0.58      0.32      0.22       995
weighted avg       0.58      0.32      0.22       995



GROUNDING [ABSENT] & PROMPT [OCR]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_OCR = constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_OCR_images = uspolitics_test_grounding_absent_prompt_with_OCR['image'].values
uspolitics_test_grounding_absent_prompt_with_OCR_entities = uspolitics_test_grounding_absent_prompt_with_OCR['entity'].values
uspolitics_test_grounding_absent_prompt_with_OCR_texts = uspolitics_test_grounding_absent_prompt_with_OCR['OCR'].values
uspolitics_test_grounding_absent_prompt_with_OCR['prediction'] = [get_prediction(image, prompt_with_OCR(entity, text)) for image, entity, text in zip(uspolitics_test_grounding_absent_prompt_with_OCR_images, uspolitics_test_grounding_absent_prompt_with_OCR_entities, uspolitics_test_grounding_absent_prompt_with_OCR_texts)]
uspolitics_test_grounding_absent_prompt_with_OCR.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_prompt[OCR]_prediction[llava-v1.5-13b].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_OCR['prediction'] = uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_OCR['prediction'] = uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_OCR['role'].values, uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_OCR['role'].values, uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

villain    676
hero       277
other       41
victim       1
Name: prediction, dtype: int64
0
0.30088631164144075
              precision    recall  f1-score   support

        hero       0.55      0.62      0.59       246
     villain       0.32      0.88      0.47       250
      victim       1.00      0.00      0.01       249
       other       0.49      0.08      0.14       250

    accuracy                           0.39       995
   macro avg       0.59      0.40      0.30       995
weighted avg       0.59      0.39      0.30       995



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [Qwen_VL_Chat]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat = constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['image'].values
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['entity'].values
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['caption'].values
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images, uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities, uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions)]
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_caption[Qwen-VL-Chat]_prompt[CAPTION]_prediction[llava-v1.5-13b].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

villain    686
hero       237
other       67
victim       5
Name: prediction, dtype: int64
0
0.3364133404353279
              precision    recall  f1-score   support

        hero       0.62      0.60      0.61       246
     villain       0.34      0.93      0.50       250
      victim       1.00      0.02      0.04       249
       other       0.46      0.12      0.20       250

    accuracy                           0.42       995
   macro avg       0.61      0.42      0.34       995
weighted avg       0.61      0.42      0.34       995



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [InternLM_XComposer]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer = constraint22_dataset_uspolitics_test_captioned_InternLM_XComposer.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_images = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['image'].values
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_entities = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['entity'].values
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_captions = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['caption'].values
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_images, uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_entities, uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_captions)]
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_caption[InternLM-XComposer]_prompt[CAPTION]_prediction[llava-v1.5-13b].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

villain    668
hero       248
other       71
victim       8
Name: prediction, dtype: int64
0
0.314318785220992
              precision    recall  f1-score   support

        hero       0.54      0.55      0.55       246
     villain       0.34      0.90      0.49       250
      victim       0.88      0.03      0.05       249
       other       0.38      0.11      0.17       250

    accuracy                           0.39       995
   macro avg       0.53      0.40      0.31       995
weighted avg       0.53      0.39      0.31       995



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [llava]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_caption_llava = constraint22_dataset_uspolitics_test_captioned_llava.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_caption_llava_images = uspolitics_test_grounding_absent_prompt_with_caption_llava['image'].values
uspolitics_test_grounding_absent_prompt_with_caption_llava_entities = uspolitics_test_grounding_absent_prompt_with_caption_llava['entity'].values
uspolitics_test_grounding_absent_prompt_with_caption_llava_captions = uspolitics_test_grounding_absent_prompt_with_caption_llava['caption'].values
uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(uspolitics_test_grounding_absent_prompt_with_caption_llava_images, uspolitics_test_grounding_absent_prompt_with_caption_llava_entities, uspolitics_test_grounding_absent_prompt_with_caption_llava_captions)]
uspolitics_test_grounding_absent_prompt_with_caption_llava.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_caption[llava-v1.5-13b]_prompt[CAPTION]_prediction[llava-v1.5-13b].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_caption_llava['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_caption_llava['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

villain    601
hero       259
other      116
victim      18
Name: prediction, dtype: int64
1
0.3427328356200932
              precision    recall  f1-score   support

        hero       0.52      0.55      0.53       246
     villain       0.34      0.82      0.48       250
      victim       0.83      0.06      0.11       249
       other       0.38      0.18      0.24       250

    accuracy                           0.40       995
   macro avg       0.52      0.40      0.34       995
weighted avg       0.52      0.40      0.34       995



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [BLIP-2]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2 = constraint22_dataset_uspolitics_test_captioned_BLIP_2.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_images = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['image'].values
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_entities = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['entity'].values
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_captions = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['caption'].values
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_images, uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_entities, uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_captions)]
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_caption[BLIP_2]_prompt[CAPTION]_prediction[llava-v1.5-13b].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

villain    723
hero       186
other       84
victim       2
Name: prediction, dtype: int64
0
0.3005226597857807
              precision    recall  f1-score   support

        hero       0.62      0.47      0.53       246
     villain       0.30      0.88      0.45       250
      victim       1.00      0.01      0.02       249
       other       0.40      0.14      0.20       250

    accuracy                           0.37       995
   macro avg       0.58      0.37      0.30       995
weighted avg       0.58      0.37      0.30       995

